In [ ]:
TIME_DIM = 4

import numpy as np
from keras.layers import MaxPooling3D, MaxPooling1D, Conv1D, RepeatVector, Lambda, Dot, Multiply, Input, Permute, LSTM, Dense, TimeDistributed, Conv2D, ConvLSTM2D, Flatten, Reshape, BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import adam
import keras.backend as k
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from keras.utils import to_categorical


d = pd.read_csv("Sandipan_compressed1.csv")

d = d.values
print(d.shape)
lab = d[:,17]
d = d[:,0:17]


x = np.zeros((d.shape[0]-TIME_DIM+1,TIME_DIM,d.shape[1]), dtype='float')
y = np.zeros((d.shape[0]-TIME_DIM+1,1), dtype='float')
SIZE = d.shape[0]-TIME_DIM+1

for i in range(SIZE):
    for j in range(TIME_DIM):
        x[i,j,:] = d[i+j,:]
    y[i] = lab[i+j]

print(x.shape)
print(np.unique(y))
y = to_categorical(y)
print(y.shape)

inp = Input(shape=(x.shape[1],x.shape[2]))
#*****Attention-1*************
perm1 = Permute((2,1))(inp)
att1 = Dense(TIME_DIM,activation='softmax')(perm1) 
perm2 = Permute((2,1))(att1)
merg1 = Multiply()([inp,perm2])
#*****END Attention-1*********

#*****Affinity Matrix*********
r_merg1 = Reshape((TIME_DIM,d.shape[1],1))(merg1)
r_perm2 = Reshape((TIME_DIM,1,d.shape[1]))(perm2)
mat_mul1 = Lambda(lambda x: tf.matmul(x[0],x[1]))([r_merg1,r_perm2])
r_mat_mul1 = Reshape((TIME_DIM,d.shape[1],d.shape[1],1))(mat_mul1)
#*****END Affinity Matrix*****

conv1 = ConvLSTM2D(64, 3, activation = 'selu',return_sequences=True)(r_mat_mul1)
bn1 = BatchNormalization()(conv1)
pool31 = MaxPooling3D((1,2,2))(bn1)
pool31 = Flatten()(pool31)
dense2 = Dense(100, activation ='selu')(pool31)
dense3 = Dense(y.shape[1], activation ='softmax')(dense2)
model = Model(inputs=inp,outputs=dense3)



model.summary()
model.compile(optimizer=adam(lr=0.00001), loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x,y,epochs=50,batch_size=2)           # Change x to x_test, y to y_test
                                                  # Change epochs as required
y1 = model.predict(x)

model.evaluate(x,y)

